In [1]:
# #Install Packages
# !pip install faiss-cpu
# !pip install sentence-transformers

In [2]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [3]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [4]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


#### Step 1 : Create source embeddings for the text column

In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text) # input is python array

In [7]:
vectors.shape

(8, 768)

In [8]:
dim = vectors.shape[1]
dim

768

#### Step 2 : Build a FAISS Index for vectors
OR
#### Step 2 : Build a Usearch Index for vectors



!pip install usearch

In [59]:
# import faiss
from usearch.index import Index

In [24]:
## Faiss
# index = faiss.IndexFlatL2(dim)

## usearch

index = Index(ndim=dim)
index

usearch.Index
- config
-- data type: ScalarKind.BF16
-- dimensions: 768
-- metric: MetricKind.Cos
-- multi: False
-- connectivity: 16
-- expansion on addition :128 candidates
-- expansion on search: 64 candidates
- binary
-- uses OpenMP: 0
-- uses SimSIMD: 0
-- supports half-precision: 1
-- uses hardware acceleration: serial
- state
-- size: 0 vectors
-- memory usage: 0 bytes
-- max level: 0
--- 0. 0 nodes

this creates empty `index`. We will add vectors to the index

#### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [28]:
??Index.add # Usearch

Signature:
Index.add(
    self,
    keys: 'KeyOrKeysLike',
    vectors: 'VectorOrVectorsLike',
    *,
    copy: 'bool' = True,
    threads: 'int' = 0,
    log: 'Union[str, bool]' = False,
    progress: 'Optional[ProgressCallback]' = None,
) -> 'Union[int, np.ndarray]'
Source:   
    def add(
        self,
        keys: KeyOrKeysLike,
        vectors: VectorOrVectorsLike,
        *,
        copy: bool = True,
        threads: int = 0,
        log: Union[str, bool] = False,
        progress: Optional[ProgressCallback] = None,
    ) -> Union[int, np.ndarray]:
        """Inserts one or move vectors into the index.

        For maximal performance the `keys` and `vectors`
        should conform to the Python's "buffer protocol" spec.

        To index a single entry:
            keys: int, vectors: np.ndarray.
        To index many entries:
            keys: np.ndarray, vectors: np.ndarray.

        When working with extremely large indexes, you may want to
        pass `copy=False`, if you

In [30]:
# index.add(vectors) # FAISS

for i, vector in enumerate(vectors):
    index.add(i, vector) # i: key and  vector: value


In [31]:
index

usearch.Index
- config
-- data type: ScalarKind.BF16
-- dimensions: 768
-- metric: MetricKind.Cos
-- multi: False
-- connectivity: 16
-- expansion on addition :128 candidates
-- expansion on search: 64 candidates
- binary
-- uses OpenMP: 0
-- uses SimSIMD: 0
-- supports half-precision: 1
-- uses hardware acceleration: serial
- state
-- size: 8 vectors
-- memory usage: 21,053,888 bytes
-- max level: 0
--- 0. 8 nodes

we can see that size is 8 vectors

In [32]:
index.size

8

#### Step 4 : Encode search text using same encorder and normalize the output vector

In [33]:
search_query = 'I want to buy a polo-shirt'

vec = encoder.encode(search_query)
vec.shape

(768,)

we need to convert the 1 dimensional array (768,) into a 2D vector of shape (1,768)

In [34]:
import numpy as np

In [35]:
svec = np.array(vec).reshape(1,-1) # svec: search vector
svec.shape

(1, 768)

In [36]:
# faiss.normalize_L2(svec)

#### Step 5: Search for similar vector in the FAISS index created

In [ ]:
## FAISS
# index.search(svec, k=2) # FAISSS return top 2 results

distances, I = index.search(svec, k=2)
distances
I
row_indices = I.tolist()[0]
row_indices

In [39]:
??Index.search

Signature:
Index.search(
    self,
    vectors: 'VectorOrVectorsLike',
    count: 'int' = 10,
    radius: 'float' = inf,
    *,
    threads: 'int' = 0,
    exact: 'bool' = False,
    log: 'Union[str, bool]' = False,
    progress: 'Optional[ProgressCallback]' = None,
) -> 'Union[Matches, BatchMatches]'
Source:   
    def search(
        self,
        vectors: VectorOrVectorsLike,
        count: int = 10,
        radius: float = math.inf,
        *,
        threads: int = 0,
        exact: bool = False,
        log: Union[str, bool] = False,
        progress: Optional[ProgressCallback] = None,
    ) -> Union[Matches, BatchMatches]:
        """
        Performs approximate nearest neighbors search for one or more queries.

        :param vectors: Query vector or vectors.
        :type vectors: VectorOrVectorsLike
        :param count: Upper count on the number of matches to find
        :type count: int, defaults to 10
        :param threads: Optimal number of cores to use
        :type t

In [41]:
index.search(svec, count=2)

usearch.Matches(4)

In [43]:
matches = index.search(svec, count=2)
matches

usearch.Matches(2)

In [48]:
matches.keys

array([3, 2], dtype=uint64)

In [47]:
matches.distances

array([0.66462904, 0.6684731 ], dtype=float32)

In [44]:
matches[0]

Match(key=np.uint64(3), distance=np.float32(0.66462904))

In [45]:
matches[0].key

np.uint64(3)

In [54]:
# or
match1, match2 = matches 

In [53]:
match1

Match(key=np.uint64(3), distance=np.float32(0.66462904))

In [57]:
row_indices = matches.keys
row_indices = row_indices.tolist()
row_indices

[3, 2]

In [56]:
row_indices.tolist()

[3, 2]

These are row indices in `df` whose rows we converted into `faiss` index

Or

For `usearch`, I used row index as key. So, it means the same thing as it was for `faiss` index

In [58]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


You can see that the two results from the dataframe are similar to a search_query

### FAISS not working

### Using Usearch